<a href="https://colab.research.google.com/github/daniel-molina23/fake-news-decision-tree/blob/daniel-develop/Model2_Updated_Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# The modules we're going to use
from __future__ import print_function
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from scipy import linalg
import numpy as np
import matplotlib.pyplot as plt
from random import seed
from random import randint

# Load data from scikit-learn, which returns (data, target)
# note: if you call "boston = load_boston()", it returns a dictionary-like object
data, target = datasets.load_boston(True)

# Split the data into two parts: training data and testing data
train_data,test_data,train_target,test_target = train_test_split(data,(target[:, np.newaxis]), test_size=0.2, random_state=42)




In [16]:
import pandas as pd

df1 = pd.read_csv('politifact_real.csv')
df2 = pd.read_csv("politifact_fake.csv")

poli_real = pd.DataFrame(df1)
poli_fake = pd.DataFrame(df2)

poli_fake.shape

(432, 4)

In [17]:
poli_real["label"] = 0
poli_fake["label"] = 1

poli_data = pd.concat([poli_real, poli_fake])

from sklearn.utils import shuffle
poli_data = shuffle(poli_data)

poli_data.head(20)

poli_data.dropna()

,id,news_url,title,tweet_ids,label
129,politifact5608,http://www.ohioart.com/etch/,Sale of the Etch A Sketch Brand,175315740583411713\t222014873649950720\t254744...,0
193,politifact13593,https://web.archive.org/web/20161226221619/htt...,"Bill Murray Had This to Say About Rochester, N...",812157455211892740\t812349255104991232\t812505...,1
100,politifact7563,https://web.archive.org/web/20050226122120/htt...,05/27/99 Committee on the Judiciary,2019233899\t2821385093\t2821982589\t5754493469...,0
380,politifact8805,http://www.guttmacher.org/statecenter/updates/...,Laws Affecting Reproductive Health and Rights:...,294479917771526144\t308929391356960770\t418786...,0
67,politifact1307,http://www.bea.gov/national/xls/gdpchg.xls,Gross Domestic Product Percent change from pre...,19899666749\t22238218943\t340314539017007104\t...,0
...,...,...,...,...,...
415,politifact14884,empirenews.net/police-discover-meth-lab-in-bac...,Police Discover Meth Lab In Back Room of Alaba...,847908880801607681\t852711031252111360\t945732...,1
309,politifact14920,https://www.ecfr.gov/cgi-bin/text-idx?gp=&SID=...,eCFR — Code of Federal Regulations,717041881948033024\t717749291251548160\t717795...,1
396,politifact15208,http://maxipolitics.com/fbi-raids-obama-find-m...,FBI RAIDS OBAMA – Find Massive Secret That Was...,961363014862721024\t961598885645955073\t977197...,1
555,politifact9438,https://web.archive.org/web/20130314224614/htt...,Everything you need to know about the drone de...,260532526676848641\t310028760328073216\t310029...,0


In [18]:
poli_data.head(30)
# com, co, gov, etc (domain parser)
# https, http, or neither (https is far more secure!)
# number of tweet id's per link
# (future, not now) using web scraping techniques to find number of advertisements per link
# poli_data.iloc[0][4]

,id,news_url,title,tweet_ids,label
204,politifact1435,https://web.archive.org/web/20081029143250/htt...,Impact of Senator Obama's Tax Proposals as Des...,NaN,0
129,politifact5608,http://www.ohioart.com/etch/,Sale of the Etch A Sketch Brand,175315740583411713\t222014873649950720\t254744...,0
193,politifact13593,https://web.archive.org/web/20161226221619/htt...,"Bill Murray Had This to Say About Rochester, N...",812157455211892740\t812349255104991232\t812505...,1
100,politifact7563,https://web.archive.org/web/20050226122120/htt...,05/27/99 Committee on the Judiciary,2019233899\t2821385093\t2821982589\t5754493469...,0
74,politifact542,NaN,Interview with Sen John McCain,771282869\t773350984\t780994481\t828178279\t86...,0
106,politifact998,https://web.archive.org/web/20090425195727/htt...,Debt Day: A Symbol of Washington’s Arrogant Cu...,NaN,0
380,politifact8805,http://www.guttmacher.org/statecenter/updates/...,Laws Affecting Reproductive Health and Rights:...,294479917771526144\t308929391356960770\t418786...,0
67,politifact1307,http://www.bea.gov/national/xls/gdpchg.xls,Gross Domestic Product Percent change from pre...,19899666749\t22238218943\t340314539017007104\t...,0
355,politifact6537,http://www.youtube.com/watch?v=8BTl2pSmpYk&fea...,Romney campaign ad,332689220617510912\t332691537249058818\t332693...,0
310,politifact15190,http://time.com/5161034/florida-school-shootin...,Student Hasn't Stopped Filming Since Florida S...,964258282356568064\t964262629672869888\t964265...,1


# Here is where we begin the webscrape process...

In [19]:
import re

# regexProtocol = re.compile(r'(http|https)?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
# regexNoProtocol = re.compile(r'[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
regexUrl = re.compile(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})')


# just for testing, uncommmment to test!
check = [
  'http://www.foufos.us',
  'https://www.foufos.gov',
  'http://foufos.gr',
  'http://www.foufos.gr/kino',
  'http://werer.uk',
  'www.foufos.org',
  'www.mp3.com',
  'www.t.co',
  'http://t.co',
  'http://www.t.co',
  'https://www.t.co',
  'www.aa.com',
  'http://aa.com',
  'http://www.aa.com',
  'https://www.aa.com',
  'www.foufos',
  'www.foufos-.gr',
  'www.-foufos.gr',
  'foufos.gr',
  'http://www.foufos',
  'http://foufos',
  'www.mp3#.com'
]

for each in check:
  if regexUrl.match(each):
    print("SUCESS:::::", each)
  else:
    print("FAILURE---------------------   ", each)


SUCESS::::: http://www.foufos.us
SUCESS::::: https://www.foufos.gov
SUCESS::::: http://foufos.gr
SUCESS::::: http://www.foufos.gr/kino
SUCESS::::: http://werer.uk
SUCESS::::: www.foufos.org
SUCESS::::: www.mp3.com
SUCESS::::: www.t.co
SUCESS::::: http://t.co
SUCESS::::: http://www.t.co
SUCESS::::: https://www.t.co
SUCESS::::: www.aa.com
SUCESS::::: http://aa.com
SUCESS::::: http://www.aa.com
SUCESS::::: https://www.aa.com
FAILURE---------------------    www.foufos
FAILURE---------------------    www.foufos-.gr
FAILURE---------------------    www.-foufos.gr
FAILURE---------------------    foufos.gr
FAILURE---------------------    http://www.foufos
FAILURE---------------------    http://foufos
FAILURE---------------------    www.mp3#.com


In [20]:
s = "\"http://www.foufos.gr/kino\" , Here is another link \"www.werer.uk\""
s1 = "try this one"
# print(regexUrl.findall(s))




def findLastIndexForDomain(s): #string
  domain = {".com/", ".org/", ".edu/", ".gov/", ".uk/", ".net/", ".ca/", ".de/", ".jp/", ".fr/", ".au/", ".us/", ".ru/", ".ch/", ".it/", ".nl/", ".se/", ".no/", ".es/", ".mil/", ".ly/", ".tel/", ".kitchen/", ".email/", ".tech/", ".estate/", ".xyz/", ".codes/", ".bargains/", ".bid/", ".expert/", ".co/", ".name/", ".mobi/", ".asia/", ".biz/", ".arpa/", ".cat/", ".jobs/", ".info/", ".int/", ".pro/", ".aero/", ".travel/", ".coop/"}
  count = s.count(".")
  dotIndex = 0
  length = len(s)
  for i in range(count):
    print("string to be examined: ", s[dotIndex:])
    index = dotIndex + s[dotIndex:].find(".") # find dot in subtring

    if(index+4 <= length): # if valid index
      domain2 = s[index:index+4]
      print(domain2)
      if domain2 in domain:
        return index+3
    if(index+5 <= length): # if valid index
      domain3 = s[index:index+5]
      print(domain3)
      if domain3 in domain:
        return index+4
    if(index+6 <= length): # if valid index
      domain4 = s[index:index+6] 
      print(domain4)
      if domain4 in domain:
        return index+5
    
    dotIndex = index + 1 # update
    print(dotIndex)


s2 = "http://www.foufos.net/kino/ufsbefui/fnieunf"
firstIndex = s2.find("www.")
lastIndex = findLastIndexForDomain(s2)
print("from ", s2, " we get...")
print(s2[firstIndex:lastIndex])


string to be examined:  http://www.foufos.net/kino/ufsbefui/fnieunf
.fou
.fouf
.foufo
11
string to be examined:  foufos.net/kino/ufsbefui/fnieunf
.net
.net/
from  http://www.foufos.net/kino/ufsbefui/fnieunf  we get...
www.foufos.net


In [ ]:
# install the depedencies
!pip install requests-HTML
!pip install urlparse4

In [21]:
'''here is where the real web scraping is performed!
  TAKES A LONG TIME (1 hr approx extract html links without same link repeat)
  Uncomment to try
'''


import re
import requests
from requests_html import HTMLSession
from urllib.parse import urlparse

# get all urls
urls = poli_data["news_url"]

data = []
for url in urls:
  if isinstance(url, float):
    print("no link present")
    data.append(url) # link is nan
    continue # go to next link
  else:
    baseUrl = urlparse(url).netloc # without the https://
    try:
      # go to webpage
      session = HTMLSession()
      r = session.get(url)
      # extract all links
      listOfLinks = r.html.xpath('//a/@href')

      numOfLinks = 0
      for link in listOfLinks:
        if baseUrl not in link and regexUrl.match(link):
          numOfLinks += 1
      data.append(numOfLinks + 1) # links + itself
    except Exception:
      print("NOT VALID LINK: ", url)
      data.append(0) #no valid links

print(data[:10])

no link present
no link present
NOT VALID LINK:  our.news/2017/09/10/breaking-clinton-foundations-sends-water-to-houstonfor-7-a-bottle/
NOT VALID LINK:  steemit.com/news/@gaucher/donald-trump-dead-from-a-fatal-heart-attack
NOT VALID LINK:  http://usleader.net/index.php/2017/05/05/breaking-ken-starrs-plane-just-disappeared-way-dc-testify-hillary/
no link present
NOT VALID LINK:  patrioticviralnews.com/articles/international-arrest-warrant-issued-for-george-soros/
NOT VALID LINK:  forums.spacebattles.com/threads/%E2%80%9Cgay-people-should-wear-specially-colored-clothes-to-warn-straight-people%E2%80%9D.362660/
NOT VALID LINK:  http://www.mass.gov/legis/laws/seslaw04/sl040149.htm
NOT VALID LINK:  www.newsweek.com/obama-not-trump-may-be-invited-royal-wedding-prince-harry-and-meghan-markle-722899
NOT VALID LINK:  http://rip.trendolizer.com/2017/04/breaking-conservative-icon-ted-nugent-killed-in-hunting-accident-1.html
NOT VALID LINK:  http://blogs.trendolizer.com/2015/01/new-york-man-wanted-

In [22]:
# To create new DataFrame with internal links and download


new_data = pd.DataFrame({
    'id': poli_data['id'],
    'news_url': poli_data['news_url'],
    'title': poli_data['title'],
    'tweet_ids': poli_data['tweet_ids'],
    'internal_links': data,
    'label': poli_data['label']
})

new_data.dropna(subset=['news_url'], inplace=True) # remove row which has nan as website

new_data.to_csv("polifact_clean_mix_enumerated.csv", index=False)

In [23]:
new_data.head(10)

,id,news_url,title,tweet_ids,internal_links,label
204,politifact1435,https://web.archive.org/web/20081029143250/htt...,Impact of Senator Obama's Tax Proposals as Des...,NaN,1.0,0
129,politifact5608,http://www.ohioart.com/etch/,Sale of the Etch A Sketch Brand,175315740583411713\t222014873649950720\t254744...,45.0,0
193,politifact13593,https://web.archive.org/web/20161226221619/htt...,"Bill Murray Had This to Say About Rochester, N...",812157455211892740\t812349255104991232\t812505...,1.0,1
100,politifact7563,https://web.archive.org/web/20050226122120/htt...,05/27/99 Committee on the Judiciary,2019233899\t2821385093\t2821982589\t5754493469...,1.0,0
106,politifact998,https://web.archive.org/web/20090425195727/htt...,Debt Day: A Symbol of Washington’s Arrogant Cu...,NaN,1.0,0
380,politifact8805,http://www.guttmacher.org/statecenter/updates/...,Laws Affecting Reproductive Health and Rights:...,294479917771526144\t308929391356960770\t418786...,22.0,0
67,politifact1307,http://www.bea.gov/national/xls/gdpchg.xls,Gross Domestic Product Percent change from pre...,19899666749\t22238218943\t340314539017007104\t...,21.0,0
355,politifact6537,http://www.youtube.com/watch?v=8BTl2pSmpYk&fea...,Romney campaign ad,332689220617510912\t332691537249058818\t332693...,3.0,0
310,politifact15190,http://time.com/5161034/florida-school-shootin...,Student Hasn't Stopped Filming Since Florida S...,964258282356568064\t964262629672869888\t964265...,25.0,1
211,politifact681,https://web.archive.org/web/20050322064340/htt...,LexisNexis(R) Publisher,395031939565887488\t641393866667483137\t665522...,1.0,0


# WebScraping done, now qualitatively refactor the data

In [24]:
# most common domains (45 total)
domain = [".com/", ".org/", ".edu/", ".gov/", ".uk/", ".net/", ".ca/", ".de/", ".jp/", ".fr/", ".au/", ".us/", ".ru/", ".ch/", ".it/", ".nl/", ".se/", ".no/", ".es/", ".mil/", ".ly/", ".tel/", ".kitchen/", ".email/", ".tech/", ".estate/", ".xyz/", ".codes/", ".bargains/", ".bid/", ".expert/", ".co/", ".name/", ".mobi/", ".asia/", ".biz/", ".arpa/", ".cat/", ".jobs/", ".info/", ".int/", ".pro/", ".aero/", ".travel/", ".coop/"]
d_encoding = [index for index in range(len(domain))] # encoded by index
print(len(domain) == len(d_encoding))

# Protocols for encryption
protocol = ["no_protocol", "http", "https"]
p_encoding = [0, 1, 2]

# Number of tweet id's per row


True


In [28]:
# data_domain method
def encodeDomain(link):
  num = len(domain) # none of the domains encoding, default
  if isinstance(link, float): # nan, therefore return
    return num
  for i in range(len(domain)):
    if domain[i] in link:
      num = i
  return num

# data_protocol method
def configProtocol(link):
  if isinstance(link, float): # nan, therefore return
    return 0
  elif link[:5] == "https":
    return 2
  elif link[:4] == "http":
    return 1
  else:
    return 0


# number of tweet id's method
def getIdCount(idCombo):
  if isinstance(ids, float): # if 'nan'
    return 0
  return len(idCombo.split('\t')) # split by tab or by the char '\t'


####################

data_protocol = []
data_domain = []
for link in new_data["news_url"]:
  data_domain.append(encodeDomain(link)) # returns domain encoding
  data_protocol.append(configProtocol(link)) # returns either 'https', 'http', or 'neither'


tweetIds = []
for ids in new_data["tweet_ids"]:
  tweetIds.append(getIdCount(ids))



label = []
for l in new_data["label"]:
  label.append(l)

capitals = []
for title in new_data['title']:
  count = 0
  for char in title:
    if char.isupper():
      count+=1
  capitals.append(count)

# are all strings the same length????
print(len(data_domain) == len(data_protocol) == len(tweetIds) == len(label)==len(capitals))


True


In [26]:
# data_domain, all numbers
# d_encoding (indices of domain)
# domain (list of string domains)
d_freq = dict()
d_set = set(domain)
for i in data_domain:
  if i == len(domain):
    if "NULL" not in d_freq:
      d_freq["NULL"] = 1
    else:
      d_freq["NULL"] += 1
    continue # go to next iteration

  if domain[i] not in d_freq:
    d_freq[domain[i]] = 1
  else: # valid
    d_freq[domain[i]] += 1

display(d_freq)

{'.co/': 4,
 '.com/': 571,
 '.edu/': 4,
 '.gov/': 119,
 '.info/': 7,
 '.mil/': 1,
 '.net/': 17,
 '.org/': 217,
 '.ru/': 1,
 '.uk/': 6,
 '.us/': 12,
 '.xyz/': 1,
 'NULL': 35}

In [85]:
data = []
for linkNum in new_data['internal_links']:
  data.append(int(linkNum))


clean_data = pd.DataFrame({
    "domain_type": data_domain,
    "protocol": data_protocol,
    "title_capital_freq": capitals,
    "tweet_ids": tweetIds,
    "internal_links": data,
    "label": label
})

# clean_data.to_csv("clean_traintest_polimix_data.csv", index=False)

In [32]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [86]:
features = ["domain_type", "protocol","title_capital_freq", "tweet_ids", "internal_links"]
X = clean_data[features]
y = clean_data.label

display(X.head())
display(y.head())

,domain_type,protocol,title_capital_freq,tweet_ids,internal_links
0,1,2,16,0,1
1,0,1,5,3,45
2,1,2,12,5,1
3,3,2,2,55,1
4,1,2,13,0,1


0    0
1    0
2    1
3    0
4    0
Name: label, dtype: int64

In [90]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1) # 80% training and 20% test

In [91]:
clf = DecisionTreeClassifier()

# train decision tree
clf = clf.fit(X_train, y_train)

# predict
y_pred = clf.predict(X_test)

In [92]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8241206030150754


#Using another type of decision Tree classifier

In [93]:
clf = DecisionTreeClassifier(criterion="entropy", max_depth=5) # the greater the max_depth, can cause overfitting

# Train DT
clf = clf.fit(X_train,y_train)
#Predict response for test dataset
y_pred = clf.predict(X_test)


print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8291457286432161
